In [ ]:
import duckdb


In [ ]:
con = duckdb.connect(database='dados_duckdb.db', read_only=False)

In [ ]:
df = con.execute("""
                 SELECT *
                 FROM (
                    SELECT *, ROW_NUMBER() OVER (PARTITION BY NATBR ORDER BY data_ingestao DESC) AS row
                    FROM bronze_z00019
                    WHERE data_ingestao >= '2024-01-20'
                 ) WHERE row = 1
                 """).fetchdf()
df.head(20)   


In [ ]:
df_final = df.drop(columns=['nome_arquivo', 'data_ingestao', 'row'])
df_final = df_final.rename(columns={
    'NATBR': 'id',
    'MAKTX': 'nm_produto',
    'WERKS': 'id_categoria',
    'MAINS': 'id_fornecedor',
    'LABST': 'vl_preco'
})

df_final.head(20)

In [ ]:
df2 = df_final
df2 = df2.astype(
    {
    'id': int,
    'nm_produto': str,
    'id_categoria': str,
    'id_fornecedor': int,
    'vl_preco': float
    }
)

In [ ]:
df2.dtypes

In [ ]:
con.execute("""
   CREATE TABLE IF NOT EXISTS produtos ( 
       id BIGINT,
       nm_produto TEXT,
       id_categoria TEXT,
       id_fornecedor BIGINT,
       vl_preco FLOAT
   )
""")

In [ ]:
df_resultado = con.execute("SELECT * FROM produtos").fetchdf()
df_resultado.head(10)

In [ ]:
con.execute("INSERT INTO produtos SELECT * FROM df2")

In [ ]:
con.close()